**Import  libraries**

In [ ]:
import torch
import numpy as np
from tqdm import tqdm,tqdm_notebook
import torchvision
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch import nn,optim
import torch.nn.functional as F
from torch.utils import data

**Loading the data**

In [ ]:
data_set = torchvision.datasets.CIFAR10(root='CIFAR10',train=True,download = True, transform = transforms.Compose([transforms.ToTensor()]))

batch_size =20

train, val, test = data.random_split(data_set,[40_000,5_000,5_000])


train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size,
    shuffle=True)
valid_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, 
    shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size,shuffle=True)

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

**Network Architecture**

In [ ]:

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1 = nn.Conv2d(3,16,3,padding=1)
    self.conv2 = nn.Conv2d(16,32,3,padding =1)
    self.conv3 = nn.Conv2d(32,64,3,padding =1)
    self.pool = nn.MaxPool2d(2,2)
    self.l1  = nn.Linear(64*4*4,512)
    self.l2  = nn.Linear(512,64)
    self.l3  = nn.Linear(64,10)
    self.dropout = nn.Dropout(p=0.3)
  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = torch.flatten(x,start_dim=1)
    x = self.dropout(x)
    x = self.dropout(F.relu(self.l1(x)))
    x = self.dropout(F.relu(self.l2(x)))
    #x = F.relu(self.l2(x))
    out = self.l3(x)
    return out
model = Net()
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum = 0.9)

**Training the network**

In [ ]:
epochs = 25
valid_loss_min = np.Inf

for epoch in tqdm(range(epochs)):
  correct = 0
  total = 0
  train_loss = 0.0
  valid_loss = 0.0
  model.train()
  for images,labels in train_loader:
    optimizer.zero_grad()
    output = model(images)
    loss = criterion(output,labels)

    predicted = torch.max(output.data, 1)[1]
    total += len(labels)
    correct += (predicted == labels).sum()

    loss.backward()
    optimizer.step()
    train_loss += loss.item()*images.size(0)

  train_accuracy = 100 * correct / float(total)

  model.eval()
  for images,labels in valid_loader:
    output = model(images)
    loss = criterion(output,labels)

    predicted = torch.max(output.data, 1)[1]
    total += len(labels)
    correct += (predicted == labels).sum()

    valid_loss += loss.item()*images.size(0)

  valid_accuracy = 100 * correct / float(total)

  train_loss = train_loss/len(train_loader.dataset)
  valid_loss = valid_loss/len(valid_loader.dataset)
  print('epoch:{}, TrainingLoss:{},ValidationLoss:{}, TrainAccuracy:{},ValidAccuracy:{}'.format(epoch+1,train_loss,valid_loss,train_accuracy,valid_accuracy))
  if valid_loss<valid_loss_min:
    valid_loss_min = valid_loss
    print('Validation loss decreased from {:.8f} to {:.8f}'.format(valid_loss_min,valid_loss))
    print("Saving model ---->")
    torch.save(model.state_dict(),'model.pt')

Loading the model

In [ ]:
model.load_state_dict(torch.load('model.pt'))

**Testing the network**

In [ ]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()

for images,labels in test_loader:
  output = model(images)
  loss = criterion(output,labels)
  test_loss += loss.item()*images.size(0)
  _, pred = torch.max(output, 1)    
  # compare predictions to true label
  correct_tensor = pred.eq(labels.data.view_as(pred))
  correct = np.squeeze(correct_tensor.numpy())
  for i in range(batch_size):
    label = labels.data[i]
    class_correct[label]+=correct[i].item()
    class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if train_on_gpu:
    images = images.cuda()

# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx] if not train_on_gpu else images[idx].cpu())
    ax.set_title("{} ({})".format(classes[preds[idx]], classes[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))